#第5章 画像分類
##第5.2節 データセットの準備

###モジュールのインポート

In [1]:
import random
import numpy as np
from PIL import Image
from typing import Sequence, Callable

import torch
import torchvision
import torchvision.transforms as T
import torchvision.transforms.functional as F

###物体検出用COCOデータセットを扱うCocoDetectionクラス

In [2]:
class CocoDetection(torchvision.datasets.CocoDetection):
    '''
    物体検出用COCOデータセット読み込みクラス
    img_directory: 画像ファイルが保存されてるディレクトリへのパス
    anno_file    : アノテーションファイルのパス
    transform    : データ拡張と整形を行うクラスインスタンス
    '''
    def __init__(self, img_directory: str, anno_file: str,
                 transform: Callable=None):
        super().__init__(img_directory, anno_file)

        self.transform = transform

        # カテゴリーIDに欠番があるため、それを埋めてクラスIDを割り当て
        self.classes = []
        # 元々のクラスIDと新しく割り当てたクラスIDを相互に変換する
        # ためのマッピングを保持
        self.coco_to_pred = {}
        self.pred_to_coco = {}
        for i, category_id in enumerate(
                sorted(self.coco.cats.keys())):
            self.classes.append(self.coco.cats[category_id]['name'])
            self.coco_to_pred[category_id] = i
            self.pred_to_coco[i] = category_id

    '''
    データ取得関数
    idx: サンプルを指すインデックス
    '''
    def __getitem__(self, idx: int):
        img, target = super().__getitem__(idx)

        # 親クラスのコンストラクタでself.idsに画像IDが
        # 格納されているのでそれを取得
        img_id = self.ids[idx]

        # 物体の集合を一つの矩形でアノテーションしているものを除外
        target = [obj for obj in target
                  if 'iscrowd' not in obj or obj['iscrowd'] == 0]

        # 学習用に当該画像に映る物体のクラスIDと矩形を取得
        # クラスIDはコンストラクタで新規に割り当てたIDに変換
        classes = torch.tensor([self.coco_to_pred[obj['category_id']]
                                for obj in target], dtype=torch.int64)
        boxes = torch.tensor([obj['bbox'] for obj in target],
                             dtype=torch.float32)

        # 矩形が0個のとき、boxes.shape == [0]となってしまうため、
        # 第1軸に4を追加して軸数と第2軸の次元数を合わせる
        if boxes.shape[0] == 0:
            boxes = torch.zeros((0, 4))

        width, height = img.size
        # xmin, ymin, width, height -> xmin, ymin, xmax, ymax
        boxes[:, 2:] += boxes[:, :2]

        # 矩形が画像領域内に収まるようにクランプ
        boxes[:, ::2] = boxes[:, ::2].clamp(min=0, max=width)
        boxes[:, 1::2] = boxes[:, 1::2].clamp(min=0, max=height)

        # 学習のための正解データを用意
        # クラスIDや矩形など渡すものが多義にわたるため、辞書で用意
        target = {
            'image_id': torch.tensor(img_id, dtype=torch.int64),
            'classes': classes,
            'boxes': boxes,
            'size': torch.tensor((width, height), dtype=torch.int64),
            'orig_size': torch.tensor((width, height),
                                      dtype=torch.int64),
            'orig_img': torch.tensor(np.asarray(img))
        }

        # データ拡張と整形
        if self.transform is not None:
            img, target = self.transform(img, target)

        return img, target

    '''
    モデルで予測されたクラスIDからCOCOのクラスIDに変換する関数
    label: 予測されたクラスID
    '''
    def to_coco_label(self, label: int):
        return self.pred_to_coco[label]

###無作為に画像を水平反転するクラス

In [3]:
class RandomHorizontalFlip:
    '''
    無作為に画像を水平反転するクラス
    prob: 水平反転する確率
    '''
    def __init__(self, prob: float=0.5):
        self.prob = prob

    '''
    無作為に画像を水平反転する関数
    img   : 水平反転する画像
    target: 物体検出用のラベルを持つ辞書
    '''
    def __call__(self, img: Image, target: dict):
        if random.random() < self.prob:
            # 画像の水平反転
            img = F.hflip(img)

            # 正解矩形をx軸方向に反転
            # xmin, xmaxは水平反転すると大小が逆転し、
            # width - xmax, width - xminとなる
            width = img.size[0]
            target['boxes'][:, [0, 2]] = width - \
                target['boxes'][:, [2, 0]]

        return img, target

###無作為に画像を切り抜くクラス

In [4]:
class RandomSizeCrop:
    '''
    無作為に画像を切り抜くクラス
    scale: 切り抜き前に対する切り抜き後の画像面積の下限と上限
    ratio: 切り抜き後の画像のアスペクト比の下限と上限
    '''
    def __init__(self, scale: Sequence[float],
                 ratio: Sequence[float]):
        self.scale = scale
        self.ratio = ratio

    '''
    無作為に画像を切り抜く関数
    img   : 切り抜きをする画像
    target: 物体検出用のラベルを持つ辞書
    '''
    def __call__(self, img: Image, target: dict):
        width, height = img.size

        # 切り抜く領域の左上の座標と幅および高さを取得
        # 切り抜く領域はscaleとratioの下限と上限に従う
        top, left, cropped_height, cropped_width = \
            T.RandomResizedCrop.get_params(
                img, self.scale, self.ratio)

        # 左上の座標と幅および高さで指定した領域を切り抜き
        img = F.crop(img, top, left, cropped_height, cropped_width)

        # 原点がx = left, y = topに移動し、合わせて矩形の座標も移動
        target['boxes'][:, ::2] -= left
        target['boxes'][:, 1::2] -= top

        # 矩形の座標が切り抜き後に領域外に出る場合は座標をクリップ
        target['boxes'][:, ::2] = \
            target['boxes'][:, ::2].clamp(min=0)
        target['boxes'][:, 1::2] = \
            target['boxes'][:, 1::2].clamp(min=0)
        target['boxes'][:, ::2] = \
            target['boxes'][:, ::2].clamp(max=cropped_width)
        target['boxes'][:, 1::2] = \
            target['boxes'][:, 1::2].clamp(max=cropped_height)

        # 幅と高さが0より大きくなる(矩形の面積が0でない)矩形のみ保持
        keep = (target['boxes'][:, 2] > target['boxes'][:, 0]) & \
            (target['boxes'][:, 3] > target['boxes'][:, 1])
        target['classes'] = target['classes'][keep]
        target['boxes'] = target['boxes'][keep]

        # 切り抜き後の画像の大きさを保持
        target['size'] = torch.tensor(
            [cropped_width, cropped_height], dtype=torch.int64)

        return img, target

###無作為に画像をリサイズするクラス

In [5]:
class RandomResize:
    '''
    無作為に画像をアスペクト比を保持してリサイズするクラス
    min_sizes: 短辺の長さの候補、この中から無作為に長さを抽出
    max_size :  長辺の長さの最大値
    '''
    def __init__(self, min_sizes: Sequence[int], max_size: int):
        self.min_sizes = min_sizes
        self.max_size = max_size

    '''
    リサイズ後の短辺と長辺を計算する関数
    min_side: 短辺の長さ
    max_side: 長辺の長さ
    target  : 目標となる短辺の長さ
    '''
    def _get_target_size(self, min_side: int, max_side:int,
                         target: int):
        # アスペクト比を保持して短辺をtargetに合わせる
        max_side = int(max_side * target / min_side)
        min_side = target

        # 長辺がmax_sizeを超えている場合、
        # アスペクト比を保持して長辺をmax_sizeに合わせる
        if max_side > self.max_size:
            min_side = int(min_side * self.max_size / max_side)
            max_side = self.max_size

        return min_side, max_side

    '''
    無作為に画像をリサイズする関数
    img   : リサイズする画像
    target: 物体検出用のラベルを持つ辞書
    '''
    def __call__(self, img: Image, target: dict):
        # 短辺の長さを候補の中から無作為に抽出
        min_size = random.choice(self.min_sizes)

        width, height = img.size

        # リサイズ後の大きさを取得
        # 幅と高さのどちらが短辺であるかで場合分け
        if width < height:
            resized_width, resized_height = self._get_target_size(
                width, height, min_size)
        else:
            resized_height, resized_width = self._get_target_size(
                height, width, min_size)

        # 指定した大きさに画像をリサイズ
        img = F.resize(img, (resized_height, resized_width))

        # 正解矩形をリサイズ前後のスケールに合わせて変更
        ratio = resized_width / width
        target['boxes'] *= ratio

        # リサイズ後の画像の大きさを保持
        target['size'] = torch.tensor(
            [resized_width, resized_height], dtype=torch.int64)

        return img, target

###PIL画像をテンソルに変換するクラス

In [6]:
class ToTensor:
    '''
    PIL画像をテンソルに変換する関数
    img   : テンソルに変換する画像
    target: 物体検出用のラベルを持つ辞書
    '''
    def __call__(self, img: Image, target: dict):
        img = F.to_tensor(img)

        return img, target

###画像を標準化するクラス

In [7]:
class Normalize:
    '''
    画像を標準化するクラス
    mean: R, G, Bチャネルそれぞれの平均値
    std : R, G, Bチャネルそれぞれの標準偏差
    '''
    def __init__(self, mean: Sequence[float], std: Sequence[float]):
        self.mean = mean
        self.std = std

    '''
    画像を標準化する関数
    img   : 標準化する画像
    target: 物体検出用のラベルを持つ辞書
    '''
    def __call__(self, img: torch.Tensor, target: dict):
        img = F.normalize(img, mean=self.mean, std=self.std)

        return img, target

###データ整形・拡張をまとめるクラス

In [8]:
class Compose:
    '''
    データ整形・拡張をまとめて適用するためのクラス
    transforms: データ整形・拡張のクラスインスタンスのシーケンス
    '''
    def __init__(self, transforms: Sequence[Callable]):
        self.transforms = transforms

    '''
    データ整形・拡張を連続して適用する関数
    img   : データ整形・拡張する画像
    target: 物体検出用のラベルを持つ辞書
    '''
    def __call__(self, img: Image, target: dict):
        for transform in self.transforms:
            img, target = transform(img, target)

        return img, target

###2つのデータ拡張から無作為にどちらかを選択して適用する関数

In [9]:
class RandomSelect:
    '''
    2種類のデータ拡張を受け取り、無作為にどちらかを適用するクラス
    transform1: データ拡張1
    transform2: データ拡張2
    prob      : データ拡張1が適用される確率
    '''
    def __init__(self, transform1: Callable, transform2: Callable,
                 prob: float=0.5):
        self.transform1 = transform1
        self.transform2 = transform2
        self.prob = prob

    '''
    データ拡張を無作為に選択して適用する関数
    img   : データ整形・拡張する画像
    target: 物体検出用のラベルを持つ辞書
    '''
    def __call__(self, img: Image, target: dict):
        if random.random() < self.prob:
            return self.transform1(img, target)

        return self.transform2(img, target)